In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
import time
import random
import pdb

### Make a list of the urls of the categories in the forum

In [20]:
user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
html_page = requests.get('https://curltalk.naturallycurly.com/', headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser')
link_list_forum_top = []
for link in soup.find_all('a'):
    link_list_forum_top.append(link.get('href'))
categories = [s for s in link_list_forum_top if (("categories" in s) and ('https' in s))]
categories[0:]

['https://curltalk.naturallycurly.com/categories/announcements',
 'https://curltalk.naturallycurly.com/categories/newbies-info',
 'https://curltalk.naturallycurly.com/categories/introductions',
 'https://curltalk.naturallycurly.com/categories/curlmart-news',
 'https://curltalk.naturallycurly.com/categories/feedback',
 'https://curltalk.naturallycurly.com/categories/c2',
 'https://curltalk.naturallycurly.com/categories/c3',
 'https://curltalk.naturallycurly.com/categories/c4',
 'https://curltalk.naturallycurly.com/categories/general-discussion-about-curly-hair',
 'https://curltalk.naturallycurly.com/categories/salon-stylist-chatter',
 'https://curltalk.naturallycurly.com/categories/transitioning',
 'https://curltalk.naturallycurly.com/categories/thin-thinning-hair',
 'https://curltalk.naturallycurly.com/categories/readers-product-reviews',
 'https://curltalk.naturallycurly.com/categories/hall-of-shame-discussion',
 'https://curltalk.naturallycurly.com/categories/hall-of-fame-discussion'

#### Get the discussion topic urls for the first x pages of the category

In [45]:
# %%time

link_listdiscussion = []
for i in range(1, 20):
    url = f'{categories[-4]}/p{i}'
    print(url)
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
     
    # Check status code for an appropriate response fromt the API
    status = html_page.status_code
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        break
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    page1_of_discussion = []
    #time.sleep(random.randint(20,45))
    for link in soup.find_all('a'):
        link_listdiscussion.append(link.get('href'))
        topics = [s for s in link_listdiscussion if (("/discussion" in s) and ('https' in s))]

print(len(topics))     
# Loop through all the topics found for each catagory        
list_for_mongo = []
count = 1
for topic in topics:
    url2 = topic
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page2 = requests.get(url2, headers = {'User-Agent': user_agent} )
    
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        break
        
    soup2 = BeautifulSoup(html_page2.content, 'html.parser')
    signatures = soup2.find_all('div', class_="Signature UserSignature userContent")
    
    for i in range(0,len(signatures)):
        sig = {}
        sig['signature'] = soup2.find_all('div', class_="Signature UserSignature userContent")[i].get_text()
        list_for_mongo.append(sig)
#     time.sleep(random.randint(1,5))
    print(count)
    count += 1
    

https://curltalk.naturallycurly.com/categories/makeup-skin-care/p1
https://curltalk.naturallycurly.com/categories/makeup-skin-care/p2
https://curltalk.naturallycurly.com/categories/makeup-skin-care/p3
https://curltalk.naturallycurly.com/categories/makeup-skin-care/p4
https://curltalk.naturallycurly.com/categories/makeup-skin-care/p5
https://curltalk.naturallycurly.com/categories/makeup-skin-care/p6
https://curltalk.naturallycurly.com/categories/makeup-skin-care/p7
https://curltalk.naturallycurly.com/categories/makeup-skin-care/p8
https://curltalk.naturallycurly.com/categories/makeup-skin-care/p9
https://curltalk.naturallycurly.com/categories/makeup-skin-care/p10
https://curltalk.naturallycurly.com/categories/makeup-skin-care/p11
https://curltalk.naturallycurly.com/categories/makeup-skin-care/p12
https://curltalk.naturallycurly.com/categories/makeup-skin-care/p13
https://curltalk.naturallycurly.com/categories/makeup-skin-care/p14
https://curltalk.naturallycurly.com/categories/makeup-ski

In [46]:
len(list_for_mongo)

4097

### Create a mongoDB and have data stored from scrape 

In [47]:
curly_collection.find({})

In [48]:
next(_)

{'_id': ObjectId('5d4761a20bd621c34eabc2b7'),
 'signature': "Animal obsessed wavy with a love for writing\xa02B/2C Indian wavy, low-medium porosity, medium-high density, coarse hairShampoo: WOW Miracle 10-in-1 ShampooConditioner: WOW Argan Oil ConditionerGel: Arata Hair GelDC/ Treatment: Shea Moisture Manuka Honey and Mafura Oil Intensive Hydration MasqueClarifying Shampoo: Pantene Lively CleanCG since May '18After nearly two decades of neglect, CG finally helped me embrace my hair as a part and parcel of my identity"}

In [49]:
# DB creation 
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
curlydb = myclient['Naturaly_Curly_db']
curly_collection = curlydb['curly_profiles']

In [50]:
entry2 = curly_collection.insert_many(list_for_mongo)

In [51]:
query_1 = curly_collection.find({})
test = []
for x in query_1:
    test.append(x['signature'])
    #print(x)
print(len(test))    

20593


In [52]:
test_set = set(test)

In [53]:
# Number of unique entries so far
len(test_set)
#1406

1752

In [11]:
query_2 = curly_collection.find({})
test2 = []
for x in query_2:
    test2.append(x['signature'])
    #print(x)
print(len(test2))  

15624


In [13]:
len(set(test2))

1406

In [14]:
test2

["Animal obsessed wavy with a love for writing\xa02B/2C Indian wavy, low-medium porosity, medium-high density, coarse hairShampoo: WOW Miracle 10-in-1 ShampooConditioner: WOW Argan Oil ConditionerGel: Arata Hair GelDC/ Treatment: Shea Moisture Manuka Honey and Mafura Oil Intensive Hydration MasqueClarifying Shampoo: Pantene Lively CleanCG since May '18After nearly two decades of neglect, CG finally helped me embrace my hair as a part and parcel of my identity",
 '2c/3a | low porosity | fine/medium | low/medium density| protein,coconut, and green tea sensitive | Modified CGMLow-Poo:\xa0 Jessicurl Hair Cleansing CreamSulfate Wash: Aveda Rosemary Mint Purifying ShampooRinse Out Conditioner: DevalCurl One ConditionLeave-In Conditioner:\xa0Curl Junkie Smoothing LotionDeep Conditioner:\xa0DevaCurl Heaven In HairCurl Enhancer: Curl Junkie Spiral LotionGel: Jessicurl\xa0Spiralicious\xa0Gel',
 'Low poo: Intelligent Nutrients (IN) Harmonic; Oribe Cleansing Creme\nRO: IN Harmonic\nLI: CJ Beauticu

In [19]:
len(pd.Series(test2).unique())

1406